## CS431/631 Big Data Infrastructure
### Winter 2019 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Hao Dong
* **ID:** 20757585

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://roegiest.com/bigdata-2019w/assignment1-431.html).

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located.   You will be using the Spark installation in the CS451 course account.   The code in the following cell tells Python how to find this Spark installation.   Before going on, execute that code (by selecting the cell and hitting 'return' while holding down the shift key).   It will take a few seconds to run, and will produce no output.

In [2]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster (since we do not have a Spark cluster in the CS student computing environment), and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Run the code in the cell below to create a Spark context.   Creating the `SparkContext` causes your Python program (running in this notebook) to prepare to run Spark jobs, and will take a few seconds to complete.  Be sure that you run this code only one time, because a single Python program may only have one active SparkContext.

In [3]:
import random
sc = SparkContext(appName="YourTest", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))

Next, let's test that your `SparkContext` has been set up properly by running some simple test code (adapted from the [Spark examples page](https://spark.apache.org/examples.html)).   This code uses a single Spark job to estimate the value of $\pi$.  `parallelize()` and `filter()` are Spark *transformations*, and `count()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take a few seconds, since a Spark job is being created and executed, and should print an estimate of $\pi$ when it finishes.   

In [4]:
import random

num_samples = 100000000

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi_estimate = 4 * count / num_samples
print(pi_estimate)

3.14151256


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the $\pi$-estimation example works.   What is the Spark job doing, and how is it used to estimate the value of $\pi$?

#### Your answer to Question 1:

*Firstly use sc.parallelize to create parallelized collections. The elements of the collection $range(0,10000000)$ are copied to form a distributed dataset that can be operated on in parallel. Once created, the distributed dataset can be operated on in parallel.*

*Then, filter() as a transformation returns a new dataset formed by selecting those elements of the source on which function inside() returns true. In inside(), two random floats $x,y$ are generated uniformly in the semi-open range [0.0, 1.0). And if $x*x + y*y < 1$ the fuction returns true. Actually, filter() does not compute its results right away. Instead, they just remember the transformations applied to the base dataset. The transformations are only computed when an action requires a result to be returned to the driver program.*

*When code executes count(), Spark breaks the computation into tasks to run on separate machines, and each machine runs both its part of the filter() to generate the new dataset which satisfies that corresponding function inside() returns true and count() as an action which returns the number of elements in the dataset. Spark returns only its answer to the driver program.*

*Finally, we can get how much the area of $\frac{1}{4}$circle whose radius is 1 accounts for of the area of the square whose side length is 1 by calculating $\frac{count}{100000000}$. This number is equal to the area of $\frac{1}{4}$circle whose radius is 1, which is $\frac{1}{4}\pi$. By this, we can get the value of $\pi$.*

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [5]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
    
    lines = sc.textFile("Shakespeare.txt")
    tokens_lines = lines.flatMap(lambda s : set(simple_tokenize(s)))
    return tokens_lines.distinct().count()

# count_distinct_tokens()

25975

---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [6]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    def line_paris(s):
        pairs = []
        line = set(simple_tokenize(s))
        for t1 in line :
            for t2 in line :
                if t1 != t2 :
                    pairs.append((t1,t2))
        return pairs
    
    lines = sc.textFile("Shakespeare.txt")
    pairs_line = lines.flatMap(lambda s : line_paris(s))
    return pairs_line.distinct().count()


# count_distinct_pairs()

1969760

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your code should report (print) the 50 highest-probability tokens, and their probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [7]:
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
    lines = sc.textFile("Shakespeare.txt")
    # get the tuples of tokens and their initiallizing corresponding count
    tokens_lines = lines.flatMap(lambda s : set(simple_tokenize(s)))\
       .map(lambda token : (token,1))
    # get the tuples of probabailities, count and token.
    line_num = lines.count()
    prob_count_token = tokens_lines.reduceByKey(lambda a,b : a + b)\
                    .map(lambda token : (token[1]/line_num , token[1], token[0]))\
                    .takeOrdered(50,key = lambda x : -x[0])
    return prob_count_token
    
# top_50_tokens_probabilities()

[(0.2009178657172255, 24604, 'and'),
 (0.19843538192686472, 24300, 'the'),
 (0.1523542765682928, 18657, 'i'),
 (0.148924529226347, 18237, 'to'),
 (0.1357526662202551, 16624, 'of'),
 (0.10844534452628657, 13280, 'a'),
 (0.09959332995802643, 12196, 'you'),
 (0.09430988583840991, 11549, 'my'),
 (0.08667461497003054, 10614, 'in'),
 (0.08630714204053634, 10569, 'that'),
 (0.07150206601447026, 8756, 'is'),
 (0.06720671577193814, 8230, 'not'),
 (0.06167012363422561, 7552, 'with'),
 (0.06039621747864574, 7396, 'me'),
 (0.05982459292165477, 7326, 'for'),
 (0.05836286726877787, 7147, 'it'),
 (0.054402325695340446, 6662, 'be'),
 (0.05246696826667102, 6425, 'this'),
 (0.05228731483447386, 6403, 'his'),
 (0.050899083767495794, 6233, 'your'),
 (0.05067043394469941, 6205, 'but'),
 (0.0474938346208496, 5816, 'he'),
 (0.046889545803459144, 5742, 'have'),
 (0.04132028940534714, 5060, 'thou'),
 (0.04015254209606559, 4917, 'as'),
 (0.03952375508337552, 4840, 'him'),
 (0.03949109082297604, 4836, 'so'),
 (0

---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also report PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [8]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    # your solution to Question 5 here
    def line_paris(s):
        pairs = []
        line = set(simple_tokenize(s))
        for t1 in line :
            for t2 in line :
                if t1 != t2 :
                    pairs.append(((t1,t2),1))
        return pairs
    
    
    lines = sc.textFile("Shakespeare.txt")
    line_num = lines.count()
    
    tokens_lines = lines.flatMap(lambda s : set(simple_tokenize(s)))\
                        .map(lambda token : (token,1))\
                        .reduceByKey(lambda a,b : a + b).collect()
    tokens_dict = dict(tokens_lines)
    
    
    def PMI_calculate(line_num,pair):
        t1 = pair[0][0]
        t2 = pair[0][1]
        co_num = pair[1]
        PMI_value = round(log(co_num * line_num / (tokens_dict[t1] * tokens_dict[t2]),10),3)
        return ((pair[0],PMI_value,co_num,tokens_dict[t1],tokens_dict[t2]))

    pairs = lines.flatMap(lambda s : line_paris(s)).reduceByKey(lambda x,y : x + y)
    pairs_threshold = pairs.filter(lambda pair : pair[1] >= threshold)\
            .map(lambda pair : PMI_calculate(line_num,pair))
    return pairs_threshold.collect()
    

# PMI(1000)

[(('that', 'the'), 0.069, 2461, 10569, 24300),
 (('the', 'that'), 0.069, 2461, 24300, 10569),
 (('to', 'the'), 0.051, 4072, 18237, 24300),
 (('to', 'and'), 0.018, 3815, 18237, 24604),
 (('the', 'to'), 0.051, 4072, 24300, 18237),
 (('the', 'and'), 0.046, 5427, 24300, 24604),
 (('and', 'to'), 0.018, 3815, 24604, 18237),
 (('and', 'the'), 0.046, 5427, 24604, 24300),
 (('the', 'be'), -0.098, 1055, 24300, 6662),
 (('be', 'the'), -0.098, 1055, 6662, 24300),
 (('all', 'the'), 0.141, 1050, 3824, 24300),
 (('the', 'all'), 0.141, 1050, 24300, 3824),
 (('all', 'and'), 0.158, 1106, 3824, 24604),
 (('and', 'all'), 0.158, 1106, 24604, 3824),
 (('this', 'of'), 0.152, 1237, 6425, 16624),
 (('of', 'this'), 0.152, 1237, 16624, 6425),
 (('my', 'and'), 0.006, 2351, 11549, 24604),
 (('and', 'my'), 0.006, 2351, 24604, 11549),
 (('to', 'be'), 0.251, 1768, 18237, 6662),
 (('be', 'to'), 0.251, 1768, 6662, 18237),
 (('it', 'and'), 0.021, 1506, 7147, 24604),
 (('and', 'it'), 0.021, 1506, 24604, 7147),
 (('he', '

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```


In [9]:
from simple_tokenize import simple_tokenize
from math import log
import random
from collections import defaultdict

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    # your solution to Question 6 here
    
#     PMI_dic = {item[0] : item[1:] for item in PMI(threshold)}

    PMI_token_dic = defaultdict(list)
    for pair in PMI(threshold):
        PMI_token_dic[pair[0][0]].append(pair)
        
    lines = sc.textFile("Stest.txt")
    tokens = lines.flatMap(lambda s : set(simple_tokenize(s))).distinct()
    tokens_randoms = []
    for i in range(0,samp_size):
        tokens_randoms.append(random.choice(tokens.collect()))
    
    result = []
    for token in tokens_randoms :
        result.append((token,PMI_token_dic[token]))
    return result
    
# PMI_one_token(10,4)

[('glutton', []),
 ('fuel', []),
 ('decease', []),
 ('how',
  [(('how', 'much'), 0.595, 68, 2080, 1018),
   (('how', 'thee'), -0.015, 50, 2080, 3048),
   (('how', 'to'), -0.047, 278, 2080, 18237),
   (('how', 'be'), 0.096, 141, 2080, 6662),
   (('how', 'then'), 0.153, 53, 2080, 2195),
   (('how', 'one'), -0.066, 25, 2080, 1714),
   (('how', 'art'), 0.366, 35, 2080, 888),
   (('how', 'can'), 0.425, 54, 2080, 1195),
   (('how', 'off'), 0.172, 12, 2080, 475),
   (('how', 'from'), -0.182, 29, 2080, 2595),
   (('how', 'a'), -0.125, 169, 2080, 13280),
   (('how', 'many'), 0.795, 60, 2080, 566),
   (('how', 'and'), -0.144, 300, 2080, 24604),
   (('how', 'my'), 0.078, 235, 2080, 11549),
   (('how', 'o'), 0.326, 91, 2080, 2532),
   (('how', 'with'), -0.104, 101, 2080, 7552),
   (('how', 'that'), -0.093, 145, 2080, 10569),
   (('how', 'show'), 0.357, 16, 2080, 414),
   (('how', 'mine'), 0.126, 26, 2080, 1144),
   (('how', 'made'), 0.153, 20, 2080, 827),
   (('how', 'the'), -0.059, 360, 2080, 243

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.